In [18]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from tkinter import Tk, filedialog
import os
from glob import glob

# Widgets for directory selection
directory_path = widgets.Text(description="Directory Path")
select_button = widgets.Button(description="Select Directory")
output = widgets.Output()

# Global variables
selected_directory = None
csv_files = []
video_files = []
h5_files = []

# Function to handle directory selection
def on_select_button_click(b):
    global selected_directory, csv_files, video_files, h5_files
    with output:
        clear_output(wait=True)
        root = Tk()
        root.attributes("-topmost", True)
        root.withdraw()  # Hide the main Tkinter window
        new_directory = filedialog.askdirectory(title="Please select a directory containing the .csv files")
        root.destroy()  # Destroy the Tkinter instance

        if new_directory:  # Check if a directory was selected
            selected_directory = new_directory
            directory_path.value = selected_directory
            print(f"You chose: {selected_directory}")
            
            video_files = glob(os.path.join(selected_directory, '**', '*.avi'), recursive=True)
            csv_files = glob(os.path.join(selected_directory, '**', '*.csv'), recursive=True)
            h5_files = glob(os.path.join(selected_directory, '**', '*.h5'), recursive=True)
            
            if csv_files:
                print(f"\nFound {len(csv_files)} .csv files in the directory and subdirectories:")
                for csv_file in csv_files:
                    print(f"- {os.path.basename(csv_file)}")
            if video_files:
                print(f"\nFound {len(video_files)} .csv files in the directory and subdirectories:")
                for video_file in video_files:
                    print(f"- {os.path.basename(video_file)}")
            if h5_files:
                print(f"\nFound {len(h5_files)} .h5 files in the directory and subdirectories:")
                for h5_file in h5_files:
                    print(f"- {os.path.basename(h5_file)}")
            else:
                print("No files found in the selected directory.")
        else:
            print("No directory selected.")

# Connect button to function and display widgets
select_button.on_click(on_select_button_click)
display(directory_path, select_button, output)

Text(value='', description='Directory Path')

Button(description='Select Directory', style=ButtonStyle())

Output()

In [20]:
import napari
import imageio
import numpy as np
import pandas as pd
from pathlib import Path

# Use the first CSV file and video file from your lists
csv_file = Path(csv_files[0])  # Use the first file in your csv_files list
video_file = Path(video_files[0])  # Use the first file in your video_files list

# Predefined labels for new points
predefined_labels = ["label1", "label2", "label3", "label4", "label5"]

# Function to load video frames as a NumPy array
def load_video_as_frames(video_path):
    reader = imageio.get_reader(video_path)  # Open the video file
    frames = [frame for frame in reader]  # Read all frames
    return np.stack(frames, axis=0)  # Convert list of frames to a NumPy array (T, H, W, C)

# Function to extract points and labels from the DataFrame
def extract_points_and_labels(df):
    points = []
    labels = []
    for frame_idx in range(len(df)):
        for bodypart in df.columns.levels[1]:  # Iterate over bodyparts
            try:
                x = df.loc[frame_idx, ('DLC_resnet50_allGeckosJan26shuffle1_800000', bodypart, 'x')]
                y = df.loc[frame_idx, ('DLC_resnet50_allGeckosJan26shuffle1_800000', bodypart, 'y')]
                points.append([frame_idx, y, x])  # Napari uses (frame, y, x) order
                labels.append(bodypart)  # Add the bodypart as a label
            except KeyError:
                continue
    return np.array(points), labels


# Load the DataFrame
df = pd.read_csv(csv_file, header=[0, 1, 2])

# Extract points and labels for Napari
points_data, labels_data = extract_points_and_labels(df)

# Load video frames
video_frames = load_video_as_frames(video_file)

# Open the video frames in Napari viewer
viewer = napari.Viewer()
viewer.add_image(video_frames, name="Video Frames")

# Add collected points with labels to the viewer
viewer.add_points(
    points_data,
    size=4,  # Adjust point size as needed
    name="Body Points",
    face_color='red',
    properties={"labels": labels_data},  # Add the labels as properties
    text={"text": "{labels}", "anchor": "upper_left", "color": "white"}  # Display labels
)

# Initialize the new Points layer
points_layer = viewer.add_points(
    [],  # Start with an empty points layer
    size=4,
    name="New Points",
    face_color="red",
    properties={"labels": []},  # Start with an empty labels list
    text={"text": "{labels}", "anchor": "upper_left", "color": "white", "size": 5}
)


print("Napari viewer initialized. Add points to the 'New Points' layer, and labels will be assigned dynamically.")


Napari viewer initialized. Add points to the 'New Points' layer, and labels will be assigned dynamically.
